<a href="https://colab.research.google.com/github/RishabhMehra/DLCVNLP/blob/main/Assignment%202/Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inception/GoogleNet implementation with Keras

### 1. Import libraries

In [1]:
import cv2 
import numpy as np 
import keras
import math
from keras.layers.core import Layer
from tensorflow.keras.models import Sequential
import keras.backend as K
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten,UpSampling2D
from keras.datasets import cifar10 
from keras import backend as K 
from keras.utils import np_utils 
#from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler

### 2. Prepare the dataset

In [2]:
num_classes = 10

def load_cifar10_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

    # Resize training images
    X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:,:,:,:]])
    X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:,:,:,:]])

    # Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train, num_classes)
    Y_valid = np_utils.to_categorical(Y_valid, num_classes)
    
    X_train = X_train.astype('float32')
    X_valid = X_valid.astype('float32')

    # preprocess data
    X_train = X_train / 255.0
    X_valid = X_valid / 255.0

    return X_train, Y_train, X_valid, Y_valid

In [3]:
X_train, y_train, X_test, y_test = load_cifar10_data(32, 32)

170508288/170498071 [==============================] - 6s 0us/step


In [4]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

### 4. Create the GoogLeNet architecture

In [5]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)


const dense = UpSampling2D(input_shape=(32, 32, 3)) #64
const dense = UpSampling2D()).apply(model1)#128
const dense = UpSampling2D()).apply(model2)#256
const output= Conv2D( filters=16,kernel_size=(33,33)).apply(model3)

In [23]:
input_layer = Input(shape=(32, 32, 3))
x= UpSampling2D()(input_layer)
x= UpSampling2D()(x)
x=UpSampling2D()(x)
#x= Conv2D( filters=16,kernel_size=(33,33))(x)
type(x)

keras.engine.keras_tensor.KerasTensor

In [24]:

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init,input_shape=(224,224,3))(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
# x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


classifier_1 = AveragePooling2D((5, 5), strides=3)(x)
classifier_1 = Conv2D(128, (1, 1), padding='same', activation='relu')(classifier_1)
classifier_1 = Flatten()(classifier_1)
classifier_1 = Dense(1024, activation='relu')(classifier_1)
classifier_1 = Dropout(0.7)(classifier_1)
classifier_1 = Dense(10, activation='softmax', name='auxilliary_output_1')(classifier_1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


classifier_2 = AveragePooling2D((5, 5), strides=3)(x)
classifier_2 = Conv2D(128, (1, 1), padding='same', activation='relu')(classifier_2)
classifier_2 = Flatten()(classifier_2)
classifier_2 = Dense(1024, activation='relu')(classifier_2)
classifier_2 = Dropout(0.7)(classifier_2)
classifier_2 = Dense(10, activation='softmax', name='auxilliary_output_2')(classifier_2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = AveragePooling2D(pool_size=(7,7), strides=1, padding='valid',name='avg_pool_5_3x3/1')(x)
x = Flatten()(x)
x = Dropout(0.4)(x)
x = Dense(1000, activation='relu', name='linear')(x)
x = Dense(10, activation='softmax', name='output')(x)

### GoogleNet model without the classfiers 1 and 2

In [25]:
model = Model(input_layer, [x], name='googlenet')

In [26]:
model.summary()

Model: "googlenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
up_sampling2d_12 (UpSampling2D) (None, 64, 64, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
up_sampling2d_13 (UpSampling2D) (None, 128, 128, 3)  0           up_sampling2d_12[0][0]           
__________________________________________________________________________________________________
up_sampling2d_14 (UpSampling2D) (None, 256, 256, 3)  0           up_sampling2d_13[0][0]           
__________________________________________________________________________________________

### GoogLeNet model after adding classifiers 1 and 2

In [27]:
model_with_classifiers = Model(input_layer, [x, classifier_1, classifier_2], name='googlenet_complete_architecture')
model_with_classifiers.summary()

Model: "googlenet_complete_architecture"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
up_sampling2d_12 (UpSampling2D) (None, 64, 64, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
up_sampling2d_13 (UpSampling2D) (None, 128, 128, 3)  0           up_sampling2d_12[0][0]           
__________________________________________________________________________________________________
up_sampling2d_14 (UpSampling2D) (None, 256, 256, 3)  0           up_sampling2d_13[0][0]           
____________________________________________________________________

### compile a network with three models

In [29]:
epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

#sgd = keras.optimizers.SGD(lr=initial_lrate, momentum=0.9, nesterov=False)
sgd = tf.keras.optimizers.SGD(
    learning_rate=initial_lrate, momentum=0.9, nesterov=False, name='SGD'
)

lr_sc = LearningRateScheduler(decay, verbose=1)

model_with_classifiers.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'],
               optimizer=sgd, metrics=['accuracy'])

history = model_with_classifiers.fit(X_train, [y_train, y_train, y_train], validation_data=(X_test, [y_test, y_test, y_test]), 
                    epochs=epochs, batch_size=256, callbacks=[lr_sc])

Epoch 1/25

Epoch 00001: LearningRateScheduler setting learning rate to 0.01.
196/196 [==============================] - 487s 2s/step - loss: 6.4685 - output_loss: 2.1918 - auxilliary_output_1_loss: 2.1232 - auxilliary_output_2_loss: 2.1535 - output_accuracy: 0.1679 - auxilliary_output_1_accuracy: 0.2029 - auxilliary_output_2_accuracy: 0.1889 - val_loss: 5.6579 - val_output_loss: 1.9744 - val_auxilliary_output_1_loss: 1.8128 - val_auxilliary_output_2_loss: 1.8707 - val_output_accuracy: 0.2553 - val_auxilliary_output_1_accuracy: 0.3478 - val_auxilliary_output_2_accuracy: 0.3266
Epoch 2/25

Epoch 00002: LearningRateScheduler setting learning rate to 0.01.
196/196 [==============================] - 421s 2s/step - loss: 5.3871 - output_loss: 1.8581 - auxilliary_output_1_loss: 1.7531 - auxilliary_output_2_loss: 1.7759 - output_accuracy: 0.3022 - auxilliary_output_1_accuracy: 0.3555 - auxilliary_output_2_accuracy: 0.3485 - val_loss: 4.8466 - val_output_loss: 1.7307 - val_auxilliary_output_1_

KeyboardInterrupt: ignored